In [1]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter

In [2]:
def get_img_names(file_path):
    # input:    relative path to .txt file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines



def generate_col_names(size, dim = 2):
    """Generate the column names in the following pattern
        [img_name, x1, y1, x2, y2 ...., x(size-1), y(size-1), pitch, yaw, ....]
        
        Inputs: 
            size -> No of landmarks 
            dim -> Default is 2 (x and y coordinates only),
                    if dim = 3 then z will be included
        
        outputs:
            Returns a list contains all the column names
    """
    
    cols_names = ['img_name']
    
    for i in range(size):
        cols_names.append(f'x{i+1}')
        cols_names.append(f'y{i+1}')
    
        if dim == 3:
            cols_names.append(f'z{i+1}')
        

    cols_names.extend(['pitch', 'yaw', 'roll',  'tdx', 'tdy', 'tdz', 'scale_factor'])
    
    return cols_names


def get_data(imgs, col_names, dim=2):
    """Get landamarks and pose parameters for each image
        
        Inputs:
            imgs -> list of all image names without  extensions
            col_names -> list of all parameter names
            dime -> Default is 2 (x and y coordinates only),
                    if dim = 3 then z will be included
        
        Output:
            Pandas DataFrame that contains image name, landamarks and pose parameters for each image
    """
    
    # each element will contain all image parametars
    # Image Name + 68 landmarks + pose parameters
    data = []
    
    for image_name in imgs:        
        # image relative path
        image = f'./AFLW2000/{image_name}.jpg'

        # load mat file
        mat = sio.loadmat(f'./AFLW2000/{image_name}.mat')

        # Get 3d landmarks - 68 points
        pt3d = mat['pt3d_68']

        # Get Pose Parameters 
        # [pitch (phi), yaw (gamma), roll(theta), tdx, tdy, tdz, scale_factor]
        pose_params = mat['Pose_Para'][0]

        # Concatenating landmarks (xy coordinates) with pose params
        landmarks_xy_angles = np.hstack((image_name, pt3d.T[:, :dim].ravel(), pose_params))
        data.append(landmarks_xy_angles)
        
        # Convert data to a dataframe
        data_df = pd.DataFrame(np.array(data))
        
        # rename Column names
        data_df.columns = col_names
        

    return data_df

def convert_cols_to_numeric(df, start_idx, end_idx):
    """Change columns between start and end index to numeric  
        
        Inputs:
        
            df -- Pandas DataFrame.
            start_idx -- index of the starting column.
            end_idx -- index of the ending column.
        
        Retruns:
            A DataFrame with the new datatypes
        
        
    """
    
    # convert numeric cols from str to int
    for i in range(start_idx, end_idx):
        df.iloc[:, i] = pd.to_numeric(df.iloc[:, i])
    
    return df


def draw_axis(img, yaw, pitch, roll, tdx=None, tdy=None, size = 100):
    # COnvert degree to radians
#     pitch = pitch * np.pi / 180
#     yaw = -(yaw * np.pi / 180)
#     roll = roll * np.pi / 180

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img


In [3]:
# image names
img_names = get_img_names('./AFLW2000/*.mat')

# landmark size
landmark_size = 68

In [4]:
# column names (x, y)
col_names_xy = generate_col_names(landmark_size, dim = 2)

# final data (x, y)
data_xy = get_data(img_names, col_names_xy, 2)



# convert numeric cols from str to int

In [5]:
# No of columns
n_cols = data_xy.shape[1]

convert_cols_to_numeric(data_xy, 1, n_cols)

,img_name,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,y67,x68,y68,pitch,yaw,roll,tdx,tdy,tdz,scale_factor
0,image00002,121.868034,167.16422,122.36761,197.56262,126.81924,225.46965,130.83179,250.71490,137.52313,...,330.70197,201.82330,324.90387,-0.399231,0.018227,0.085676,231.465320,262.06660,-102.793600,0.001587
1,image00004,281.238160,265.07935,277.33942,287.04456,274.87695,307.05414,269.52377,324.18700,257.78827,...,300.59860,197.53447,302.28650,0.470065,1.189533,0.300959,330.819700,173.19424,-44.367504,0.001181
2,image00006,236.385100,239.68213,239.19826,259.59912,242.56638,277.08728,243.97137,294.20282,242.82819,...,304.68173,215.19235,304.63104,-0.184650,0.881137,-0.236852,301.050320,200.05144,-48.590317,0.001041
3,image00008,168.029220,256.93787,177.69751,277.58800,190.64160,295.48962,201.39530,311.75183,211.95721,...,319.49860,232.17296,320.84400,-0.175379,0.299208,-0.373374,262.805050,211.20130,-102.680016,0.001294
4,image00010,280.462250,236.18921,287.24982,259.10037,293.89246,279.60974,297.53937,297.52856,295.67700,...,312.92075,209.94115,312.05835,-0.882169,1.198003,-1.033374,323.511200,218.55278,-21.119057,0.001247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,image04358,112.578930,195.49637,113.78786,226.17693,119.25926,254.74318,125.33600,278.63210,133.63794,...,324.51850,209.76845,322.08582,-0.197102,-0.070430,0.105118,217.692690,226.29518,-114.195280,0.001546
1996,image04363,111.117966,243.29533,113.64708,268.56238,115.32871,291.56384,120.56592,312.71277,134.10947,...,317.92230,241.84242,318.16846,-0.232617,-1.418751,0.175960,103.658450,191.36888,-5.541234,0.001287
1997,image04364,308.227420,235.56845,309.90533,260.81784,311.43225,283.91660,310.48860,304.59222,303.66240,...,315.05124,211.82465,313.33337,-1.447955,1.431267,-1.509418,336.144870,204.93741,-20.417292,0.001289
1998,image04365,106.611570,208.57224,106.08998,237.55162,107.32884,264.50000,109.02394,287.97876,116.21605,...,313.58185,235.73334,312.26965,-0.420465,-1.191176,0.451515,110.517296,219.57965,-31.363330,0.001396


In [6]:
# column names (x, y, z)
col_names_xyz = generate_col_names(landmark_size, dim = 3)

# final data (x, y, z)
data_xyz = get_data(img_names, col_names_xyz, 3)



In [7]:
# No of columns
n_cols = data_xyz.shape[1]

convert_cols_to_numeric(data_xyz, 1, n_cols)

,img_name,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,x68,y68,z68,pitch,yaw,roll,tdx,tdy,tdz,scale_factor
0,image00002,121.868034,167.16422,-65.108890,122.36761,197.56262,-73.032080,126.81924,225.46965,-81.247860,...,201.82330,324.90387,9.237297,-0.399231,0.018227,0.085676,231.465320,262.06660,-102.793600,0.001587
1,image00004,281.238160,265.07935,-120.258255,277.33942,287.04456,-113.066240,274.87695,307.05414,-105.202910,...,197.53447,302.28650,0.852241,0.470065,1.189533,0.300959,330.819700,173.19424,-44.367504,0.001181
2,image00006,236.385100,239.68213,-96.273860,239.19826,259.59912,-93.754654,242.56638,277.08728,-89.725310,...,215.19235,304.63104,18.613464,-0.184650,0.881137,-0.236852,301.050320,200.05144,-48.590317,0.001041
3,image00008,168.029220,256.93787,-111.598220,177.69751,277.58800,-107.077736,190.64160,295.48962,-102.609350,...,232.17296,320.84400,34.102440,-0.175379,0.299208,-0.373374,262.805050,211.20130,-102.680016,0.001294
4,image00010,280.462250,236.18921,-106.988500,287.24982,259.10037,-101.327780,293.89246,279.60974,-93.347626,...,209.94115,312.05835,-8.034685,-0.882169,1.198003,-1.033374,323.511200,218.55278,-21.119057,0.001247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,image04358,112.578930,195.49637,-71.514404,113.78786,226.17693,-73.480286,119.25926,254.74318,-76.440570,...,209.76845,322.08582,51.865340,-0.197102,-0.070430,0.105118,217.692690,226.29518,-114.195280,0.001546
1996,image04363,111.117966,243.29533,95.023520,113.64708,268.56238,90.800290,115.32871,291.56384,84.510890,...,241.84242,318.16846,12.531554,-0.232617,-1.418751,0.175960,103.658450,191.36888,-5.541234,0.001287
1997,image04364,308.227420,235.56845,-115.265500,309.90533,260.81784,-111.355100,311.43225,283.91660,-104.058210,...,211.82465,313.33337,-32.042725,-1.447955,1.431267,-1.509418,336.144870,204.93741,-20.417292,0.001289
1998,image04365,106.611570,208.57224,75.812990,106.08998,237.55162,75.338290,107.32884,264.50000,71.760956,...,235.73334,312.26965,27.291553,-0.420465,-1.191176,0.451515,110.517296,219.57965,-31.363330,0.001396


## Draw Landmarks

In [18]:
# No of images
n_imgs = data_xy.shape[0]

# random index from 0 to 2000 (size of available images)
idx = np.random.randint(0, n_imgs + 1)

# get image name
img_name = data_xy.iloc[idx,0]

# relative path of the image
image_path = f'./AFLW2000/{img_name}.jpg'

# Get image info from 'data_xy' table
i1 = data_xy.iloc[idx]
img = cv2.imread(image_path)

# draw 68 landmark points
for n in range(1, landmark_size):
    x = i1[f'x{n+1}']
    y = i1[f'y{n+1}']
    
    cv2.circle(img, (int(x), int(y)), 1, (0, 255, 0), 2)

cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Draw Axis

In [19]:

# get image name
img_name = data_xy.iloc[idx,0]

# relative path of the image
image_path = f'./AFLW2000/{img_name}.jpg'

# Get image info from 'data_xy' table
i1 = data_xy.iloc[idx]

# get nose point (origin of the three axis)
nosex, nosey = data_xy[['x30', 'y30']].iloc[idx, :].to_numpy()

# get pitch, yaw, roll 
pitch, yaw, roll = data_xy.iloc[idx][['pitch', 'yaw', 'roll']].to_numpy()

# read image
img = cv2.imread(image_path)

# draw axis on image
img = draw_axis(img, -yaw, pitch, roll, nosex, nosey) 

# Display image
cv2.imshow("image", )
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save Data to Csv

In [14]:
# Create a directory 'Data' if it doesnot exists 
out_dir = './Data'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# Save data to csv
data_xy.to_csv(f'{out_dir}/Data_xy.csv',  index=False)
data_xyz.to_csv(f'{out_dir}/Data_xyz.csv',  index=False)
